#  Функции потерь (loss functions)

Функция потерь измеряет точность нейронной сети, оценивая расхождение между предсказанными результатами и истинными значениями. Она принимает два аргумента:

- Вектор истинных значений.
- Вектор предсказанных значений.

Для успешного обучения с использованием градиентного спуска функция потерь должна быть дифференцируемой и ограниченной снизу.
Рассмотрим наиболее часто применяемые функции потерь и то, как устроены их реализации в PyTorch и как ими пользоваться.

[[doc] 🛠️ Функции потерь в PyTorch](https://pytorch.org/docs/stable/nn.html#loss-functions)

##  Mean Squared Error

Средняя квадратическая ошибка (MSE) — популярная функция потерь для задач регрессии, где модель предсказывает вещественные числа.

Для $i$-го объекта выборки, если выходной вектор состоит из $C$ компонент, средняя квадратическая ошибка между выходом модели $\hat{y}$ и целевым вектором $y$ будет равна

$$\large \text{MSE}_i\left(\hat{y},y\right)=\frac{1}{C} \sum_{k=1}^{C}{\left(\hat{y}_{ik}-y_{ik}\right)^2}$$



**Примечание, относящееся ко всем функциям потерь в PyTorch, которые будут рассмотрены ниже.**

При вычислении по всему набору данных (или по мини-батчу) из $N$ объектов ошибка на отдельных объектах может усредняться или суммироваться. За это отвечает параметр `reduction`, который принимает значения `'mean'` или `'sum'`, а также может принимать значение `'none'`, при котором агрегация производиться не будет, и тогда функция будет возвращать тензор значений размером $N$.

**По умолчанию в большинстве функций потерь в PyTorch `reduction='mean'`.**


[[doc] 🛠️ MSE Loss в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss):
```python
torch.nn.MSELoss()
```

In [ ]:
import torch
from torch import nn

criterion = nn.MSELoss()

# batch of 1 element and 3 components in output vector
model_output = torch.Tensor([0.5, -0.25, 0.75])
print(f"model_output: {model_output}")

target = torch.Tensor([1, 0.25, 0.25])
print(f"target: {target}")

loss_mse = criterion(model_output, target)
print(f"loss_mse: {loss_mse}")

model_output: tensor([ 0.5000, -0.2500,  0.7500])
target: tensor([1.0000, 0.2500, 0.2500])
loss_mse: 0.25


- **Преимущество:** MSE штрафует за большие ошибки в предсказаниях, так как ошибки возводятся в квадрат.

- **Недостаток:** MSE сильно акцентируется на выбросах, что может быть нежелательно для задач, где важнее высокая точность на большинстве примеров, а не устранение одиночных выбросов.

##  Mean Absolute Error

Средняя абсолютная ошибка (MAE) также применяется для задач регрессии. В отличие от MSE, которая акцентируется на крупных выбросах (ошибка $L_2$), MAE, будучи $L_1$ ошибкой, имеет другие свойства и может быть более устойчивой к выбросам.

Для $i$-го объекта выборки, если выходной вектор состоит из $C$ компонент, средняя абсолютная ошибка между выходом модели $\hat{y}$ и целевым вектором $y$ будет равна:

$$\large \text{MAE}_i\left(\hat{y},y\right)=\frac{1}{C} \sum_{k=1}^{C}{| \hat{y}_{ik}-y_{ik}|}$$


[[doc] 🛠️ MAE Loss в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.L1Loss.html#torch.nn.L1Loss):
```python
torch.nn.L1Loss()
```

In [ ]:
criterion = nn.L1Loss()

# batch of 1 element and 3 components in output vector
model_output = torch.Tensor([0.5, -0.25, 0.75])
print(f"model_output: {model_output}")

target = torch.Tensor([1, 0.25, 0.25])
print(f"target: {target}")

loss_mae = criterion(model_output, target)
print(f"loss_mae: {loss_mae}")

model_output: tensor([ 0.5000, -0.2500,  0.7500])
target: tensor([1.0000, 0.2500, 0.2500])
loss_mae: 0.5


- **Преимущество:** MAE учитывает все ошибки равнозначно и менее чувствительна к выбросам по сравнению с MSE.

- **Недостаток:** Использование MAE может привести к большим ошибкам на отдельных примерах, поскольку модель фокусируется на общей точности, а не на устранении выбросов.

## Huber Loss

Huber Loss комбинирует среднюю квадратическую ошибку (MSE) и среднюю абсолютную ошибку (MAE). Она применима к задачам регрессии и используется для улучшения устойчивости модели к выбросам. Huber Loss ведёт себя как MSE для небольших ошибок и как MAE для больших ошибок, что позволяет сбалансировать точность и устойчивость. Выбор порога, при котором происходит переключение между MSE и MAE, задаётся гиперпараметром $\delta$.

<center><img src="https://ml.gan4x4.ru/msu/dev-2.2/L05/out/mse_mae_huber_losses.png" width="750"></img></center>

Для $i$-го объекта выборки, если выходной вектор состоит из $C$ компонент, Huber Loss между выходом модели $\hat{y}$ и целевым вектором $y$ будет равна:

$$\large \text{Huber Loss}_i(\hat{y},y)=\frac{1}{C} \sum_{k=1}^{C}{l_{ik}}$$

где

$$\large
l_{ik} =
\begin{cases}
\frac{1}{2} \left(\hat{y}_{ik}-y_{ik}\right)^2, & \text{if}\ \  |\hat{y}_{ik}-y_{ik}| < \delta \\
\delta \left(|\hat{y}_{ik}-y_{ik}| - \frac{1}{2} \delta\right), & \text{else}
\end{cases}
$$



[[doc] 🛠️ Huber Loss в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.HuberLoss.html):
```python
torch.nn.HuberLoss()
```

In [ ]:
criterion = nn.HuberLoss()

# batch of 1 element and 3 components in output vector
model_output = torch.Tensor([0.5, -0.25, 0.75])
print(f"model_output: {model_output}")

target = torch.Tensor([1, 0.25, 0.25])
print(f"target: {target}")

huber_loss = criterion(model_output, target)
print(f"huber_loss: {huber_loss}")

model_output: tensor([ 0.5000, -0.2500,  0.7500])
target: tensor([1.0000, 0.2500, 0.2500])
huber_loss: 0.125


Huber Loss комбинирует преимущества MSE и MAE и позволяет сбалансировать точность и устойчивость модели к выбросам, но требует выбора значения гиперпараметра $\delta$.

##  Cross-Entropy

Кросс-энтропия — это функция потерь, используемая для измерения несовпадения между двумя вероятностными распределениями, и она широко применяется в задачах классификации. Кросс-энтропия как функция потерь оценивает различие между истинными метками классов и предсказаниями модели.

Для $i$-го объекта выборки, если выходной вектор состоит из $C$ компонент (**логитов** для $C$ классов), кросс-энтропия между выходом модели $\hat{y}$ и целевым вектором $y$ будет равна

$$\large \text{CE}_i(\hat{y},y)= - \sum_{k=1}^{C}{y_{ik}\cdot\log\left(\frac{\exp(\hat{y}_{ik})}{\sum_{j=1}^{C}\exp(\hat{y}_{ij})}\right)}$$


[[doc] 🛠️ Cross-Entropy Loss в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss):
```python
torch.nn.CrossEntropyLoss()
```

Обратите внимание, что Cross-Entropy Loss в PyTorch уже включает в себя Softmax и принимает в качестве выхода модели логиты. Поэтому при использовании данной фукции потерь ставить на последнем слое нейронной сети Softmax **не нужно**.

In [ ]:
criterion = nn.CrossEntropyLoss()


# fmt: off
model_output = torch.tensor([[2.4, 1.9, 7.3],
                             [9.5, 2.7, 4.0],
                             [5.7, 4.1, 0.2]])  # logits
# fmt: on

print(f"model_output:\n {model_output}")

target = torch.tensor([2, 0, 1], dtype=torch.long)  # class labels
print(f"target: {target}")

loss_ce = criterion(model_output, target)
print(f"loss_ce: {loss_ce}")

model_output:
 tensor([[2.4000, 1.9000, 7.3000],
        [9.5000, 2.7000, 4.0000],
        [5.7000, 4.1000, 0.2000]])
target: tensor([2, 0, 1])
loss_ce: 0.6014580726623535


$$\text{CE}_1 = - \log\left(\frac{\exp{(7.3)}}{\exp{(2.4)}+\exp{(1.9)}+\exp{(7.3)}}\right)$$

$$\text{CE}_2 = - \log\left(\frac{\exp{(9.5)}}{\exp{(9.5)}+\exp{(2.7)}+\exp{(4.0)}}\right)$$

$$\text{CE}_3 = - \log\left(\frac{\exp{(4.1)}}{\exp{(5.7)}+\exp{(4.1)}+\exp{(0.2)}}\right)$$

$$\text{CE} = \frac{1}{3}(\text{CE}_1 + \text{CE}_2 + \text{CE}_3)$$

In [ ]:
import numpy as np

ce_1 = -np.log(np.exp(7.3) / (np.exp(2.4) + np.exp(1.9) + np.exp(7.3)))
ce_2 = -np.log(np.exp(9.5) / (np.exp(9.5) + np.exp(2.7) + np.exp(4.0)))
ce_3 = -np.log(np.exp(4.1) / (np.exp(5.7) + np.exp(4.1) + np.exp(0.2)))

ce = (1 / 3) * (ce_1 + ce_2 + ce_3)
print(f"hand-calculated loss_ce: {ce}")

hand-calculated loss_ce: 0.601458161156722


- **Преимущество:** Кросс-энтропия может учитывать веса классов, что делает её полезной для работы с несбалансированными датасетами.

- **Недостаток:** Кросс-энтропия может быть чувствительна к шуму в данных, что может привести к переобучению, особенно если модель делает сильные, но ошибочные предсказания.

### Веса классов

При работе с несбалансированными данными модель может сосредоточиться на доминирующем классе, игнорируя признаки объектов. Например, в датасете с 95% объектов класса 1 и 5% — класса 0, модель может просто присваивать всем объектам класс 1, достигая 95% точности.

Чтобы учесть дисбаланс, можно модифицировать функцию потерь, увеличивая штраф за ошибки в минорных классах. В PyTorch параметр `weight` в `CrossEntropyLoss` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) позволяет задать веса для каждого класса, корректируя функцию потерь с учетом дисбаланса.

Посмотрим, как это работает. Допустим, мы получили от нейросети неверные предсказания: второй объект должен относиться к классу $1$, а не $0$:

In [ ]:
# fmt: off
# Scores for batch of two samples
model_output = torch.tensor([[30.0, 2.0],
                             [30.0, 2.0]])

target = torch.tensor([0, 1])  # Second sample belongs to class 1
# but logit for class 0 is greater: 30 > 2. So it was misclassified
# fmt: on

Подсчитаем Cross-Entropy Loss без весов:

$$\large \text{CE} = \frac{1}{2} \biggr[- \log\frac{e^{30}}{e^{30}+e^{2}} - \log\frac{e^{2}}{e^{30}+e^{2}}\biggr]\approx 14.0 $$

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(model_output, target)
print(f"Loss = {loss.item():.2f}")

Loss = 14.00


Если у нас есть два класса с соотношением $4:1$, можно задать веса `weight = [0.2, 0.8]`. И, так как сеть ошиблась на классе с большим весом, ошибка вырастет:

$$\large \text{CE}_{\text{weighted}} =  \biggr[\mathbf{-0.2} \log\frac{e^{30}}{e^{30}+e^{2}} -\mathbf{0.8} \log\frac{e^{2}}{e^{30}+e^{2}}\biggr]\approx 22.4 $$

In [ ]:
weights = torch.tensor([0.2, 0.8])
criterion = torch.nn.CrossEntropyLoss(weight=weights)
loss = criterion(model_output, target)
print(f"Loss = {loss.item():.2f}")

Loss = 22.40


Сумма весов не обязана быть равной единице, важно только соотношение между весами. Часто соотношение весов делают обратно пропорциональным количеству объектов в классах.

In [ ]:
criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor([1.0, 4.0]))
loss = criterion(model_output, target)
print(f"Loss = {loss.item():.2f}")

Loss = 22.40


- Иногда качество модели можно улучшить, взяв квадратные корни от полученных таким образом весов (немного снижает штрафы за ошибки на редких классах).

- Несмотря на интуитивно понятную логику работы способа, он не всегда дает значительный эффект. Тем не менее, на практике стоит пробовать экспериментировать с этим способом наряду с прочими техниками борьбы с дисбалансом.

##  Focal Loss

Focal Loss — это функция потерь, используемая в нейронных сетях для решения проблемы классификации *сложных* объектов (hard examples).

[[colab] 🥨 Подробное рассмотрение Focal Loss с примерами](https://colab.research.google.com/drive/1rM7zRySu8WulXbFiXzxBGVzILxvQ6K4A)


<center><img src="https://ml.gan4x4.ru/msu/dev-2.2/L05/out/hard_examples_fruits.png" width="800"></img></center>

Focal Loss немного модифицирует кросс-энтропию для придания большей значимости ошибкам на сложных объектах.

Она была предложена в статье [Focal Loss for Dense Object Detection (Lin et al., 2017) 🎓[arxiv]](https://arxiv.org/abs/1708.02002) изначально для задачи детектирования объектов на изображениях. Определяется так:

$$\large\text{FL}\left(p_t\right) = -\left(1 - p_t\right)^\gamma\log\left(p_t\right),$$

где $p_t$ — предсказанная вероятность истинного класса, а $\gamma\geq0$ — настраиваемый гиперпараметр.

Focal Loss уменьшает потери на уверенно классифицируемых примерах (где $p_t>0.5$) и больше фокусируется на сложных примерах, которые классифицированы неправильно. Параметр $\gamma$ управляет относительной важностью неправильно классифицируемых примеров. Более высокое значение $\gamma$ увеличивает важность неправильно классифицированных примеров. В экспериментах авторы показали, что параметр $\gamma=2$ показывал себя наилучшим образом в их задаче.

При $\gamma=0$ Focal Loss становится равной Cross-Entropy Loss, которая может быть выражена как обратный логарифм вероятности истинного класса:

$$\large\text{CE}\left(p_t\right)=-\log\left(p_t\right)$$

Focal Loss может применяться также и в задачах с дисбалансом классов. В этом смысле объекты преобладающего класса могут считаться простыми, а объекты минорного класса — сложными.

Однако для работы с дисбалансом в Focal Loss могут быть добавлены веса для классов. Тогда формула будет выглядеть так:

$$\large\text{FL}\left(p_t\right) = -\alpha_t\left(1 - p_t\right)^\gamma\log\left(p_t\right),$$

где $\alpha_t$ — вес для истинного класса, имеющий такой же смысл, как параметр `weight` в Cross-Entropy Loss.

Focal Loss не реализована в PyTorch нативно, но существуют сторонние совместимые реализации. Посмотрим, как воспользоваться [одной из них 🐾[git]](https://github.com/AdeelH/pytorch-multi-class-focal-loss).

In [ ]:
#!wget -qN https://raw.githubusercontent.com/AdeelH/pytorch-multi-class-focal-loss/master/focal_loss.py

!wget -qN https://ml.gan4x4.ru/msu/dev-2.1/L05/focal_loss.py

In [ ]:
from focal_loss import FocalLoss


criterion = FocalLoss(alpha=None, gamma=2.0)

# fmt: off
model_output = torch.tensor([[2.4, 1.9, 7.3],
                             [9.5, 2.7, 4.0],
                             [5.7, 4.1, 0.2]])  # model output is logits, as in CrossEntropyLoss
# fmt: on
print(f"model_output:\n {model_output}")

target = torch.tensor([2, 0, 1], dtype=torch.long)  # class labels
print(f"target: {target}")

loss_fl = criterion(model_output, target)
print(f"loss_fl: {loss_fl}")

model_output:
 tensor([[2.4000, 1.9000, 7.3000],
        [9.5000, 2.7000, 4.0000],
        [5.7000, 4.1000, 0.2000]])
target: tensor([2, 0, 1])
loss_fl: 0.4129861891269684


Стоит отметить, что **Focal Loss следует применять с осторожностью**: если мы имеем **ошибки в разметке**, то при большом $\gamma$ можно начать очень сильно наказывать модель за ошибки на неверно размеченных примерах, что может привести к переобучению под ошибки в разметке.

## Negative Log Likelihood


NLL Loss, используемая для классификации, отличается от Cross-Entropy Loss тем, что ожидает логарифмы вероятностей классов на выходе модели, а не логиты.

Для $i$-го объекта выборки, если выходной вектор состоит из $C$ компонент (**логарифмов вероятностей** для $C$ классов), NLL Loss между выходом модели $\hat{y}$ и целевым вектором $y$ будет равна:

$$\large \text{NLL}_i\left(\hat{y},y\right)= - \sum_{k=1}^{C}{y_{ik}\cdot\hat{y}_{ik}}$$


Для применения NLL Loss в классификации логиты модели нужно сначала преобразовать через Softmax и взять их натуральный логарифм. В PyTorch это выполняется с помощью модуля `LogSoftmax` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.LogSoftmax.html). Связь между NLL Loss и Cross-Entropy Loss можно выразить так:

<center><img src="https://ml.gan4x4.ru/msu/dev-2.2/L05/out/ce_loss_vs_nll_loss.png" width="900"></center>

Исторически NLL Loss использовалась с LogSoftmax, тогда как Cross-Entropy Loss объединяет LogSoftmax и NLL Loss, упрощая модель. Cross-Entropy Loss стала популярнее за счёт упрощения.

После обучения модели:

- **LogSoftmax + NLL Loss:** На выходе логарифмы вероятностей. Для получения вероятностей нужно взять экспоненту.
  
- **Cross-Entropy Loss:** На выходе логиты. Для получения вероятностей примените Softmax.

[[blog] ✏️ Объяснение Negative Log Likelihood Loss](https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/)

[[blog] ✏️ О соотношении Cross-Entropy Loss и Negative Log Likelihood Loss](https://jamesmccaffrey.wordpress.com/2020/06/11/pytorch-crossentropyloss-vs-nllloss-cross-entropy-loss-vs-negative-log-likelihood-loss/)

##  Binary Cross-Entropy

В частном случае, когда количество классов равно двум (**задача бинарной классификации**), их можно закодировать одним числом: $0$ — для первого класса, и $1$ — для второго. Тогда сумму $\displaystyle \sum_{k=1}^{C}$ в формуле Cross-Entropy Loss можно расписать в явном виде.

Для $i$-го объекта выборки, когда выход модели является скаляром (**вероятностью** отнесения объекта к классу $1$), бинарная кросс-энтропия между выходом модели $\hat{y}$  и целевым значением $y$ будет равна

$$\large \text{BCE}_i\left(\hat{y},y\right)= - \left[{y_i\cdot\log\left(\hat{y_i}\right)+\left(1-y_i\right)\cdot\log\left(1-\hat{y_i}\right)}\right]$$

[[doc] 🛠️ Binary Cross-Entropy Loss в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html#torch.nn.BCELoss):

```python
torch.nn.BCELoss()
```



Важной особенностью BCE Loss является то, что здесь используется не one-hot кодирование целевых значений для двух классов, а **одно число: 0 — первый класс, 1 — второй класс.** При этом значения целевой переменной должны быть представлены как вещественные (float) числа.

In [ ]:
criterion = nn.BCELoss()

model_output = torch.rand(1)
print(f"model_output: {model_output}")

target = torch.empty(1).random_(2)
print(f"target: {target}")

loss_bce = criterion(model_output, target)
print(f"loss_bce: {loss_bce}")

model_output: tensor([0.5254])
target: tensor([1.])
loss_bce: 0.6436772346496582


Если классы "абсолютно полностью" не совпали, то возникает ситуация взятия логарифма от $0$, а он не определён и стремится к бесконечности, поэтому берётся "обрезанная бесконечность", равная $100$.

In [ ]:
criterion = nn.BCELoss()

model_output = torch.ones(5)
print(f"model_output: {model_output}")

target = torch.zeros(5)
print(f"target: {target}")

loss_bce = criterion(model_output, target)
print(f"loss_bce: {loss_bce}")

model_output: tensor([1., 1., 1., 1., 1.])
target: tensor([0., 0., 0., 0., 0.])
loss_bce: 100.0


## Binary Cross-Entropy With Logits

Для бинарной классификации выход модели должен быть вероятностью принадлежности к классу 1, которую можно получить с помощью логистической функции (sigmoid) и передать в BCE Loss.

BCE With Logits Loss объединяет две операции:

- Применение функции sigmoid.
- Расчёт BCE Loss.

BCE With Logits Loss ожидает логиты на входе, и её использование упрощает модель.

<center><img src="https://ml.gan4x4.ru/msu/dev-2.2/L05/out/bce_loss_vs_bce_with_logits_loss.png" width="900"></center>

Функции потерь BCE Loss и BCE With Logits Loss можно применять не только в случае бинарной классификации, но и **в случае Multi-label классификации**.

В таком случае в векторе целевых значений вероятностей для каждого объекта может быть больше, чем одна единица. А логиты на последнем слое сети независимо пропускаются через сигмоиду (а не через Softmax, как в случае с многоклассовой классификацией).



В PyTorch доступны другие функции потерь, и можно создавать собственные.

[[blog] ✏️ Обзор функций потерь в PyTorch с примером написания своей собственной функции (custom loss function)](https://neptune.ai/blog/pytorch-loss-functions)